In [ ]:
!pip install dataframe_image

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta
import dataframe_image as dfi
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import os
import pyarrow as pa
import pyarrow.parquet as pq

pd.set_option('mode.chained_assignment',None)

In [5]:
pasta = '/content/Apoio'
ap_fat = pd.DataFrame()
for diretorio, subpastas, arquivos in os.walk(pasta):
    for arquivo in arquivos:
        file_loc = os.path.join(os.path.realpath(diretorio), arquivo) #2min 14s
        ap_fat = ap_fat.append(pd.read_excel(io=file_loc, header=6))

In [6]:
ap_fat.to_excel('apoio-fat.xlsx') # 3m 25s

In [7]:
apoio_fat = ap_fat[['Ciclo', 'Rota', 'Matrícula', 'Nome do Cliente',
       'Tipo de Cliente', 'Categoria Principal', 'Subcategoria',
       'Res', 'Com', 'Ind', 'Pub', 'Qtde. Economias', 'Situação Água',
       'Situação Esgoto', 'Situação HD', 'Nº HD',
       'Cons. Estimado', 'Cons. Medido Água', 'Cons. Faturado Água', 'Cons. Medido Esgoto',
       'Cons. Faturado Esgoto', 'Cons. Faturado Esgoto FA', 'Cons. Pipa',
       'Cons. Resíduo Subst.', 'Cons. Resíduo Corte', 'Cons. Resíduo Aferição',
       'Crédito Utilizado', 'Média Faturada 12 Meses', 
       'Tipo Retenção', 'Situação Retenção','Motivo Retenção', 'Tarifa',
       'Dt. Faturamento', 'Dt. Emissão',
       'Dt. Vencimento', 'Vl. Água', 'Vl. Esgoto',
       'Vl. Esgoto Fonte Alternativa', 'Vl. Recursos Hídricos', 'Vl. Juros',
       'Vl. Multa', 'Vl. Parcelamentos', 'Vl. Serviços', 'Vl. Devolução',
       'Vl. Desconto', 'Vl. Desconto Pq. Comércio',
       'Vl. Desconto Residencial Social', 'Vl. Desconto Estimadas', 'Vl. ICMS',
       'Vl. ASEP', 'Vl. Imposto Retido', 'Vl. Total', 'Vl. Total ',
       'Em Débito Automático?']]

print(f"{round(ap_fat['Vl. Total'].sum(), 2)}, {round(ap_fat['Vl. Água'].sum(), 2)}, {round(ap_fat['Vl. Esgoto'].sum(), 2)}")

48795151.23, 26252391.39, 20816934.9


In [8]:
ap_fat.drop(['Leitura Anterior', 'Leitura Atual', 'Conta Corrente'], axis=1).to_parquet(
    './apoio-fat.parquet',engine='pyarrow', compression='gzip') #1s

In [9]:
tstt = pd.read_parquet('/content/apoio-fat.parquet') #0s

In [10]:
# tstt['Dt. Vencimento'].strftime('%m')
tstt = pd.read_parquet('/content/apoio-fat.parquet')
tstt['Dt. Faturamento'] = tstt['Dt. Faturamento'].apply(lambda linha: linha.strftime('%m-%y'))
mes_fat = tstt[['Dt. Faturamento', 'Matrícula']].groupby('Dt. Faturamento').agg('count').index.to_series().max()

ref = mes_fat.split('-')

if int(ref[0]) <= 10:
  if int(ref[0]) == 1:
    referencia = "12" + "-" + str(int(ref[1]) - 1)
  else:
    referencia = "0" + str(int(ref[0]) - 1) + "-" + ref[1]
else:
  referencia = str(int(ref[0]) - 1) + "-" + ref[1]



In [13]:
tstt[['Dt. Faturamento', 'Matrícula']].groupby('Dt. Faturamento').agg('count')

,Matrícula
Dt. Faturamento,
03-22,1
04-22,104229
